In [1]:
import os
from pymongo import MongoClient
from bson import ObjectId
from dotenv import load_dotenv
import openai
from openai import OpenAI

In [2]:
# Load environment variables
load_dotenv()
openai.api_key = os.getenv("OPEN_AI")
MONGO_URI = os.getenv("MONGO_URI")
DATABASE_NAME = "project_workflow"

In [3]:
# Initialize MongoDB client
def get_db():
    client = MongoClient(MONGO_URI)
    return client[DATABASE_NAME]

# Utility: Save data to the database
def save_to_db(collection_name, data):
    db = get_db()
    collection = db[collection_name]
    result = collection.insert_one(data)
    return str(result.inserted_id)

# Utility: Retrieve data by ID
def get_from_db(collection_name, doc_id):
    db = get_db()
    collection = db[collection_name]
    return collection.find_one({"_id": ObjectId(doc_id)})

In [4]:
# Initialize project state in the database
def initialize_project():
    project_state = {
        "context": "",
        "questions_asked": 0,
        "stories": None,
        "parsed_stories": None,
        "selected_story": None,
        "user_flow": [],
        "agent_responses": {},
        "tasks": {}
    }
    return save_to_db("project_states", project_state)

In [5]:
# Update project state in the database
def update_project_state(state_id, updates):
    db = get_db()
    collection = db["project_states"]
    collection.update_one({"_id": ObjectId(state_id)}, {"$set": updates})

In [6]:
client = OpenAI(
    api_key=os.environ.get("OPENAI_API_KEY"),  # This is the default and can be omitted
)


def query_gpt(prompt,model="gpt-4o-mini", max_tokens=500, temperature=0.7):
    """
    Query the OpenAI GPT model with a given prompt.
    Args:
        prompt (str): The input prompt for the GPT model.
    Returns:
        str: The generated response from the GPT model.
    """
    response = client.chat.completions.create(
        model=model,  # or another appropriate model
        messages=[
            {"role": "system", "content": "You are a helpful assistant."},
            {"role": "user", "content": prompt}
        ]
    )
    return response.choices[0].message.content

In [7]:
# Ask a question and update context
def ask_question(state_id, custom_question=None):
    state = get_from_db("project_states", state_id)
    
    if state["questions_asked"] == 0:
        question = "What kind of application do you want to build? Please provide a detailed description."
    elif custom_question:
        question = custom_question
    else:
        prompt = f"Based on this context, ask a clarifying question:\n\n{state['context']}"
        question = query_gpt(prompt)

    answer = input(f"{question}\nYour response: ")
    context_update = f"Q{state['questions_asked'] + 1}: {question}\nA{state['questions_asked'] + 1}: {answer}\n"

    state["context"] += context_update
    state["questions_asked"] += 1
    state["user_flow"].append({"question": question, "answer": answer})

    update_project_state(state_id, state)
    return state_id

In [8]:
# Generate user stories
def generate_stories(state_id):
    state = get_from_db("project_states", state_id)
    prompt = f"""Based on this context, generate 5 distinct user stories.
    Format each story exactly like this:

    [STORY_TITLE]: <creative theme or say or expression based on the context of the user story>
    [USER_TYPE]: <type of user this story is about>
    [USER_NEED]: <3-4 sentences what the user wants to accomplish>
    [ACCEPTANCE_CRITERIA]: <key requirements for the story to be complete>
    [VALUE]: <business value or user benefit>

    Context:
    {state['context']}"""

    stories = query_gpt(prompt, max_tokens=800)
    parsed_stories = parse_stories(stories)

    state["stories"] = stories
    state["parsed_stories"] = parsed_stories
    update_project_state(state_id, state)
    return state_id

In [9]:
# Parse stories into structured format
def parse_stories(stories_text):
    stories = []
    current_story = {}

    for line in stories_text.split('\n'):
        line = line.strip()
        if not line:
            if current_story:
                stories.append(current_story)
                current_story = {}
            continue

        for tag in ['STORY_TITLE', 'USER_TYPE', 'USER_NEED', 'ACCEPTANCE_CRITERIA', 'VALUE']:
            if line.startswith(f'[{tag}]:'):
                current_story[tag] = line.split(':', 1)[1].strip()
                break

    if current_story:
        stories.append(current_story)

    return stories

In [10]:
# Generate development plans
def generate_development_plans(state_id):
    state = get_from_db("project_states", state_id)
    agent_types = ['frontend', 'backend', 'design', 'product', 'ai']
    responses = {}
    tasks = {}

    for agent_type in agent_types:
        prompt = f"As a {agent_type} developer, what are the key components needed for the project?\n\nContext:\n{state['context']}"
        response = query_gpt(prompt)
        responses[agent_type] = response
        tasks[agent_type] = extract_tasks(response)

    state["agent_responses"] = responses
    state["tasks"] = tasks
    update_project_state(state_id, state)
    return state_id

In [11]:
# Extract tasks from a response
def extract_tasks(response):
    tasks = []
    for line in response.split('\n'):
        line = line.strip()
        if line.startswith(('-', '*', '1.', '2.', '3.')) or ':' in line:
            tasks.append(line.lstrip('-*123456789. ').strip())
    return tasks

In [12]:
state_id = initialize_project()
print(f"Project initialized with ID: {state_id}")

Project initialized with ID: 6790b10de182da0b007e115b


In [17]:
state_id = ask_question(state_id)

In [18]:
state_id = generate_stories(state_id)

In [19]:
state_id = generate_development_plans(state_id)

In [20]:
state_id

'6790b10de182da0b007e115b'

In [22]:
state = get_from_db("project_states", state_id)
for flow in state["user_flow"]:
    print(flow)




{'question': 'What kind of application do you want to build? Please provide a detailed description.', 'answer': 'Mepro saas APP'}
{'question': 'Could you provide more details about the specific features and functionalities you envision for the Mepro SaaS app?', 'answer': 'uath flow and log in '}
{'question': 'What specific authentication flows and login features are you thinking about for the Mepro SaaS app, such as single sign-on, two-factor authentication, or social media login options?', 'answer': 'two-factor authentication'}
{'question': 'Could you clarify how you would like the two-factor authentication to be implemented in the Mepro SaaS app? For example, are you considering using SMS codes, authenticator apps, or email verification?', 'answer': 'sms codes and face recognition'}
{'question': 'Could you elaborate on how you would like the face recognition feature to work alongside the SMS codes for two-factor authentication in the Mepro SaaS app? For instance, would you like users

In [24]:
print(state["stories"])


1. [STORY_TITLE]: **Secure Gateway to Mepro**
   [USER_TYPE]: **New User**
   [USER_NEED]: As a new user, I want to register for the Mepro SaaS app, so that I can access the features I need. I need a simple and intuitive login process that includes two-factor authentication for added security. This way, I feel confident that my account is protected from unauthorized access.  
   [ACCEPTANCE_CRITERIA]: The registration process must be straightforward, allowing users to sign up using email and password; the login process should include mandatory SMS code verification after entering credentials; and face recognition should be offered as an alternative or additional authentication method.  
   [VALUE]: This enhances user trust in the security of the Mepro SaaS app, increasing user adoption and engagement.

2. [STORY_TITLE]: **The Safeguard of Access**
   [USER_TYPE]: **Returning User**
   [USER_NEED]: As a returning user, I want to log into my Mepro account using two-factor authentication,

In [32]:
for strory in state["parsed_stories"]:
    print(strory)
    print("==========================")
    print(state["tasks"]["frontend"])



{'USER_TYPE': '**New User**', 'USER_NEED': 'As a new user, I want to register for the Mepro SaaS app, so that I can access the features I need. I need a simple and intuitive login process that includes two-factor authentication for added security. This way, I feel confident that my account is protected from unauthorized access.', 'ACCEPTANCE_CRITERIA': 'The registration process must be straightforward, allowing users to sign up using email and password; the login process should include mandatory SMS code verification after entering credentials; and face recognition should be offered as an alternative or additional authentication method.', 'VALUE': 'This enhances user trust in the security of the Mepro SaaS app, increasing user adoption and engagement.'}
['For a frontend developer building the Mepro SaaS app with the specified features, a range of key components will be needed to ensure a robust and user-friendly application. Here’s a detailed outline of the essential components:', '###

In [30]:
state["agent_responses"]["frontend"]

'For a frontend developer building the Mepro SaaS app with the specified features, a range of key components will be needed to ensure a robust and user-friendly application. Here’s a detailed outline of the essential components:\n\n### 1. **User Interface Components:**\n   - **Login Page:**\n     - Fields for username/email and password.\n     - "Forgot Password" link.\n     - Buttons for logging in and signing up.\n   - **Two-Factor Authentication (2FA) Flow:**\n     - Screen for entering SMS codes.\n     - Screen for face recognition (using device camera).\n     - Loading indicators for authentication processes.\n   - **Error Handling:**\n     - Display message components for:\n       - Incorrect credentials.\n       - Invalid or expired SMS codes.\n       - Face recognition failures or issues.\n\n### 2. **Authentication Components:**\n   - **Authentication Service:**\n     - Service for managing login sessions.\n     - APIs for handling SMS codes and verifying face recognition.\n   